---
title: A critical evaluation of RNNs in Spam classification
author: Christopher Gundler
---

# Introduction

In our modern society, text intelligence has become a crucial tool used in a broad number of applications. Even if our language is a somewhat complicated system of rules and facts, current algorithms are nowadays able to extract a remarkable amount of useful information out of such kind of data. The fundamental concept, allowing us to use those techniques in a wide range of tasks like security analysis, information extraction or the construction of complex classification system is commonly summarized under the term "big data." It is not only the available and continually rising computational power giving us the capabilities to find, process and interpret the enormous amount of data with evermore advanced algorithms. For the first time in the history of the humanity, we have this amount of data available in fractions of seconds; our shared knowledge is no longer hidden in libraries used by few privileged, but utterly available around the globe through the internet.

However, without the suitable algorithms, even these available datasets are just wasted bytes until we can obtain knowledge out of them. Besides classical machine learning algorithms, in the context of the growth of "Deep Learning" techniques in the last decades, new architectures were developed especially optimized for Text intelligence. A family of those are the Recurrent Neural Networks, whose have the unique advantage of using not only the plain features as the foundation of there prediction but even their sequence as additional features. The aim of this paper is the critical evaluation of such models in comparison to other Python-based state-of-the-art technology. The benchmark of choice is the commonly used task: All algorithms will classify comments according to their nature as real expressions of opinions or mostly unwanted messages like advertisements, often referred to as "Spam." Besides a focus on an optimal structure, this analysis will focus on the performance of the network in relation to the required preprocessing to conclude at the end, if the performance of recurrent neural networks is significantly better than comparable algorithms.

# Method

## Datasets

Every data mining algorithm is only that good as the data, which is feed into it. Having, therefore, a suitable number of reliable and correctly classified samples for training is crucial. In this experiment, 1956 comments are used for the generation of the model and its evaluation. These samples were extracted and hand-annotated by Alberto and Team [@alberto2015] using five highly popular videos on "YouTube" and contain beside the author, the date of writing and an id the actual comment and classification label. Other than comparable dataset[@o2012], all the 1005 spam comments and 951 non-spam comments are written in English. While this fact implies already a loss of potential useable knowledge, it simplifies the actual processing of the data dramatically: Beside some emoticons, the texts do not utilize advanced and therefore hardly handleable Unicode features; some comments contain additional makeup data for formatting purposes in the form of HTML tags. The data is split into five comma-separated CSV files, where quotes are used to escape comma in the actual text.
Due to its nature, trained data mining models should have the ability to correctly classify data which vary from those they were trained on. A common problem is the so-called "overfitting": In this case, the algorithm learns not model behind the actual features but the features itself. To minimize this risk, a separation of training and testing data is therefore essential; in this report, a ratio of 70% training and 30% testing data was used. Moreover, most machine learning algorithm contains a partly random element leading to different models between different training sets. To find a reliable average of performance, all the following experiments were applied at least three times. The actual data was used to create four different setups provides not only information of the actual generalizability of a model but also about the differences in performance with a different amount of samples. 

| ID | Name           | Training samples | Testing samples | Description                                                         |
|----|----------------|:----------------:|:---------------:|---------------------------------------------------------------------|
| 1  | Single         | 245              | 105             | Training and testing on all samples of a single file                |
| 2  | SingleSplitted | 350              | 105             | Training on all samples of a single file, testing on 30% of another |
| 3  | Multi          | 1369             | 587             | Training and testing on all samples of all files                    |
| 4  | MultiSplitted  | 1606             | 350             | Training on four files, testing on the other                        |

On the datasets 1 and 3, a classical 3-fold cross validation where used, guaranteeing that the algorithm trains all the time on different data. To archive that goal, the sets were split into three parts of equal length preserving the original distribution of the classes. The training was applied afterward on the two parts and tested on the other while shifting through the set.
On the sets 2 and 4, another approach was applied: After the selection of the training and testing set the data was shuffled after each evaluation. While such a strategy may not provide such a solid overview of the general performance as a cross-validation, it is useful to evaluate the independence of an algorithm towards the actual order of the samples instead of their underlying features.

## Preprocessing

Other than many other data mining tasks, Text intelligence is applied to unstructured data rather than structured data one could imagine as a table mapping samples to the value of their features. It is essential to understand the difference between both representations and the steps needed to generated knowledge out of them. 
In text and vision intelligence, the actual sample is not represented by ordered columns of features containing a precisely defined datatype required by the majority of algorithms. Instead, it is, in the beginning, a plain sentence; an ordered bunch of different words most likely rather different from the deterministic and unique nature numbers are. Even worse, the type of text in this experiment itself is more comparable with Twitter tweets rather than reliable and mostly error-free sources like or medical reports often used as the foundation of natural language processing tasks. In order of archiving a good performance in the evaluation, extensive preprocessing is necessary to standardize the data in a format an algorithm may understand. It is a valid way of imaging this processing from the raw data over the vectorization until the generation of the model as a pipeline with input and output, in this experiment utilizing the well known "scikit-learn"[@scikit] and "nltk" [@nltk] packages.

### Step 1: Loading and Tokenization

To allow a flexible approach to the choice of the data sources, the setups mentioned above were constructed as instances of an abstract class "Data," encapsulating common logic like the actual parsing of the files and providing a convenient way to access training and test datasets. Due to the nature of the experiment, the other given metadata like the name of the author or time of the comment was not included in the model generation.
After the actual process of loading the CSV files, the Unicode characters of no further interest and the HTML tags were removed to prepare the data for the following steps. While the actual tokenization, the conversion from a single string into a list of words, requires in the typical case just a splitting at whitespace, the noisy nature of the comments requires a more elaborate strategy in the generation of further usable entities. For being able to process common internet slang like URLs, emoticons and simple ASCII art like arrows, the TwitterTokenizer of the NLTK package was applied instead. The resulting list of entities was used afterward as the starting point for further processing.

### Step 2: Preprocessing

For a convenient integration into the Scikit-learn workflow, the different word preprocessors were designed as subclasses of an overall "Preprocessor" subclass. Afterwards, the different algorithms were integrated into a single Scikit-learn "Transformer" allowing their activation and inactivation using boolean switches.

Preprocessing 1: Standardization
In the first preprocessing step, standardization was performed to reduce the amount of lexically different but semantical equal entities significantly. This process is essential for two reasons: On the one hand, it extends the raw input with semantic information usable for the classification algorithm. As an example, many spam comments contain URL or YouTube channels being advertised. While humans can quickly identify these entities through the presence of the characteristic parts "http" and (often) ".com," every one of these looks completely different for an algorithm. On the other hand, further preprocessing algorithms like POS taggers may be irritated by the special chars used in the "noisy" way we humans use social media. Utilizing different regular expressions, URLs, numbers, and emoticons were replaced by a fixed constant in this step. Afterwards, any other special character beside the expected punctuation got removed.

#### Preprocessing 2: Slang removal
In this step, the extensive amount of slang words and spelling mistakes got reduced. For doing that, first multiple times repeated character like in the word "loooooove" got removed. Because almost any English term contains at most two times the same letters behind each other, the overall amount of proper English words were untouched by replacing only three or more occurrences. In a next step, common slang words and spelling mistakes got replaced with their proper form according to the dictionary generated by Han and his team [@han]. Even if this list was generated automatically and is not entirely trustworthy; this step reduced the amount of variance in the text again.

#### Preprocessing 3: POS tagging and lemmatization
In the next step, all the words in the now somewhat cleaned sentences got reduced to their standard form. The algorithm of choice was, in this case, the Stanford Lemmatizer. Other than a slightly simple Stemmer, this algorithm utilized additional knowledge about the actual word it is applied to improve the actual result. For providing the exact type of word, the entities were POS-tagged according to the sentence structure utilizing the Penrose corpus as their corp. After the classification by the Perceptron Tagger provided by the NLTK package, this knowledge was used to lemmatize the comment word by word.

#### Preprocessing 4: Stop word removal
In this step, every word known for its minimal information value got removed after their consideration for the actual sentence structure in the POS-tagging step before. While words like "to" may be essential and commonly used in our natural way of communication, they do not carry any significant data for the actual classification of the elements. The deletion is, therefore, an easy and efficient way to reduce again the number of words the algorithms have to handle. 

#### Preprocessing 5: Stemming
Often, Stemming is used as a less complicated alternative to a lemmatization. Instead of reducing a word to a valid English term for a phrase, its reduction to a word stem usually is more straightforward to implement. Nevertheless, the usage besides the lemmatization has a reason: The combination of both approaches should lead to an optimal reduction of features being misspelled or otherwise wrongly classified. At the end of this step, the comments consisted out of a list of not longer syntactic correct words.

#### Preprocessing 6: Lowercase
In the next step, the capitalization of all words was standardized to a complete lowercase variant. After the tagging and stemming, the kind of the first character did not carry needed knowledge anymore. By lowercasing all the characters, same words perceived by a human were also equal in the byte representation the text algorithm are applied on.

#### Preprocessing 7: Finalization
In the final step, the last remaining punctation necessary in the former steps like dots, commas, and hyphens got removed. Only the actual lowercase characters remained and represented the source of features used in the vectorization and classification of the following algorithms.

### Step 3: Vectorization

Neither Recurrent Neural Networks nor any other data mining algorithm works on real words. While these are convenient to use for humans and allow us to communicate quickly, the different encodings and sizes would result in an inefficient way of processing them for a computer. Even for us humans, it seems reasonable, that a representation where each distinct concept matches a different word is an optimized solution. Therefore, every machine learning algorithm used in text processing needs a function which can uniquely transform the words into a binary representation. The actual implementation of such a function may differ and is an implementation detail. Beside a classical "Bag of Word"-approach, where each word is represented merely by a unique number getting incremented through its calculation, fast hash algorithms with low potential of hash collision may be used for the same task, eliminating the necessity of storing an actual list of already founded words. More advanced methods like "word2vec" [@mikolov2014] even consider the similarity of words.

In the following experiment, after the actual vectorization of the preprocessed words using a bag of words containing at most 2000 words, two different formats were used to store in that way generated data. The reason is the entirely different type of input RNNs needs in comparison to other algorithms due to their fundamental differences. While former just take a vector of values with a fixed length matching the number of words to be considered sequential, other machine learning algorithms get a (dense) vector with the length of all words as input, where an element different zero marks this word as "used." The  Python library "scikit-learn" provides an already developed class for the latter task; for the RNN, a custom Bag of word matching the above-stated requirements was developed.

## Algorithms

Recurrent neural networks are, like the name already included, a specific part of the family of artificial neural networks. With the increasing popularity of feed-forward neural networks in the last decades due to their often proven excellent performance in a wide range of problems, RNNs were conducted as a more powerful tool especially for tasks involving the use of sequences of data rather than only data itself. To understand their specialty, it is essential to understand the difference between them and classical feed-forward neuronal networks. While former generates a fixed output vector out of a fixed input vector and a fixed number of computational steps, RNNs work on sequences of these vectors. Foundation of this is their ability to have an interior state which gets adapted between different samples and allows therefore further consideration of spatial frequency and common pattern. These abilities were successfully applied in speech recognition, machine translation and even the generation of text. For a rough evaluation of the performance of a "typical" RNN in the classification of Spam in this experiment, a rather trivial architecture with only a single hidden layer was used. To reduce the source of possible mistakes and improve the performance of the network, the python toolkit Keras [@chollet2015] was utilized for the description of the topology using the highly optimized Tensorflow framework as its backend. 

Unlike the other data mining tools, the input was not handled as a dense vector with a length of the complete number of words, but as a 16 number long sequence of the actual word indices. Longer sentences were truncated, shorter comments padded with zeros. In the first Embedding layer, each integer of the input vector was itself turned into its vector representation. After this transformation, an optimal Dropout layer was added before the actual artificial neurons: RNNs tend to easily overfit the data, by adding a certain amount of random noise it is possible to reduce this danger and improve the prediction results; the actual amount given in percentage was designed to be one of the hyperparameters. 
The following layer with actual RNN neurons afterward and their numbers were designed flexibly, too: Besides the classical, simple RNN neurons described above, LSTM and GRU neurons were evaluated in this experiment. Both were designed to face the "long-term dependency problem": While RNNs can find dependencies between adjacent elements easily, further context commonly needed in language processing tasks are not possible. By adding additional complexity to the actual neurons LSTMs [@hochreiter1997] may deal with that kind of "long-time memory." GRUs [@gru], target the same problem but with a simplified model with less complexly. Even if studies show both types of advanced cells perform somewhat similar, it seems worth to explore if the GRUs may be better in dealing with the rather small amount of samples provided in the experiment.
After the following Dropout layer, the results were directly mapped to a single neuron of a Dense layer used as output; due to the nature of the binary classification, the sigmoid function was utilized as its activation function. For the Backpropagation as part of the training process and under respect of the binary output of the Dense layer, Keras' "binary_crossentropy" beside the "Adam" optimizer with the default parameter described in its paper was applied.

Beside the neuronal network architecture, two other families of classifier were used to generate a valid ground truth for the evaluation. 
Random forests are one of the best performing algorithms in the family of decision trees. Inspired by the human reasoning process, those trees classify upon a sequential ordered number of (often binary) decisions. While this approach is often practical, it has, in general, a problem with overfitting [@randomforest]. Random forests try to minimize the problem by evaluating not a single, but a specific amount of individual trees and generate its prediction according to the predictions generated by the different trees comparable to a meta-classifier.
Naive Bayes is the the third and last utilized algorithm. Even if it assumes conditional independence between the features, these family of algorithm performs surprisingly well on the classification tasks. Different members like Multinomial Naive Bayes, binarized Multinomial Naive Bayes and Bernoulli Naive Bayes focus on different types of features; in the following, Multinomial Naive Bayes is used according to its focus on multiple occurrences of a word for a classification [@naivebayes].

# Results

## Part 1: Evaluation on unprocessed data

As a preparation for the evaluation of the different classifier regarding their performance, a run without any preprocessing was performed. The gained results were not only be used to generate a measurement for the dependency of the algorithms regarding preprocessing, but also to gain first insights into the valid range of favorable parameters. Most of the algorithms are highly dependent on their hyperparameters; the optimal set depends itself on the actual data there are applied on. A commonly used try-and-error method to find such parameters without knowing suitable heuristics is the Grid search: By a systematic generation and evaluation of all possible combination of parameters, this method is slow but guarantees to find a global optimum. To increase the reproducibility of the results, this evaluation is performed multiple times; the results of the evaluation are afterward averaged. In the following analysis, the F-Measure is used to describe this ability of the model to classify unseen data. Unlike the commonly used accuracy considering only the rate of detected true-positives, this measurement combines precision and recall and is therefore far better suitable for contexts, where the ratio between the classes is not exactly equal. While the term "recall" describes the capability of an algorithm to correctly classify all its targets in a set,  "precision" states how correct the algorithm was when it classified a sample as belonging to its target. 
In the following table, the averaged F-scores, precisions, and recalls of the best performing model on the different datasets were summarized. Since all scores were averaged independently from each other, the precision and recall may not result in the presented F-Score value but may be used to evaluate the performance in more detail. The variance describes the diversity of the F-Score between the three trials the analysis was performed and states therefore how the generated model depends on the distribution of the samples in training set.

| Type  | Dataset       | F-Score | Variance | Precision | Recall |
| ----- | ------------- | ------- | -------- | --------- | ------ |
| Bayes | Single        | 81.4%   | +/- 5.8% | 83.6%     | 82.9%  |
|       | Splitted      | 76.4%   | -        | 71.7%     | 73.7%  |
|       | Multi         | 89.8%   | +/- 2.1% | 89.5%     | 89.4%  |
|       | MultiSplitted | 84.5%   | -        | 84.7%     | 83.4%  |
| RF    | Single        | 81.6%   | +/- 1.9% | 86.1%     | 82.9%  |
|       | Splitted      | 68%     | +/- 0.5% | 64.3%     | 65.7%  |
|       | Multi         | 89%     | +/- 1.8% | 90.3%     | 89.7%  |
|       | MultiSplitted | 87.1%   | +/- 0.2% | 86.8%     | 86.8%  |
| RNN   | Single        | 77.7%   | +/- 8.7% | 77.1%     | 76.9%  |
|       | Splitted      | 67.9%   | +/- 5%   | 67.8%     | 68%    |
|       | Multi         | 88.9%   | +/- 5.6% | 88.6%     | 88.4%  |
|       | Multisplitted | 81.1%   | +/- 6.3% | 80.5%     | 78.9%  |

With an average F-score from 84.35% considering all datasets, the Random Forest algorithm performed in this part of the experiment slightly better than the Naive Bayes approach with its 83.025%; the RNN followed with 78.27% performance. Due to two reasons, it is difficult to find a clear winner after that round: On the one hand, the measured difference in performance at least between Random Forest and Naive Bayes is statistically insignificant. On the other hand, the variance up to 9% in the calculation leads to a rather high risk of getting results which are not reproducible. Typically, we may use two different methods utilizing the "law of the large numbers" to reduce the risk: One may increase the amount of data to reduce the influence of a single sample towards the prediction or repeat the experiments multiple time to get a Gaussian distribution of performance. Unfortunately, in this case, none of both methods is applicable due to the constraints regarding the datasets and the performance of available computers. It is, therefore, meaningful to have a look at the performance of the four datasets to find tendencies allowing further research.

When being applied to a small, single dataset, the Random Forest algorithm performed best. Foundation of this claim is not the again statistical insignificant better performance of 81.6% in comparison to the 81.4% derived by Naive Bayes but the actual variance. Almost three times smaller than the variance of Naive Bayes, the value indicates the general stability of the model over different training sets essential in production environments. In general, it seems that the Random forest was better in classifying "real" spam (as shown by the high precision) rather than find them among all the other comments (measured by the recall). In absolute terms, the RNN performed worst at this dataset. Not only the rather small F-score shows the inappropriateness of the algorithm for the task, but the high variance between the different training sets. It seems it was in general not able to find the typical pattern underlying the data but learned more the unstable noise of the training data.

The performance on the split dataset, where the model was trained on the comments below one video for classifying the spam under another one, differed highly. This types of tasks are interesting under the premise that they measure the generalizability of a model even regarding unseen data from a different source; a situation, similar to those the algorithms have to face in real-world applications. Unlike the first dataset, the training data was only shuffled and not part of a cross-validation. This time, Naive Bayes performed regularly and unquestionably best. The entirely missing variance can prove the determinism of its performance. Therefore, the actual interpretation of Multinomial Naive Bayes seems to be independent of the order of the samples. The Random Forest algorithm and the RNN performed with 68% very similar. For interpreting the visible gap in the performance of former, one may consider the common problem of tree-based algorithms with overfitting: It seems that the extracted decisions used for the classification seem to utilize more the noise in the training data than the model behind the concept "Spam." The extreme variance in the performance of the RNN, based solely on the different order of the samples, shows the inability of this technique to extract useful patterns for classification on small datasets.

The 3-fold cross-validated dataset containing 70% of all available samples for training allowed the algorithms to develop their full strength. On the unprocessed data, all algorithms were capable of showing a reasonable performance around 89%. Like it would be expected, the increased amount of available samples reduced the significance of unique samples and led therefore to a lower variance on all models. Nevertheless, the actual benefit in comparison to the first dataset depended highly on the algorithm: While Naive Bayes profited highly and the RNN shows a significant drop, too, the results for the Random Forest algorithm were fairly subtle.

Surprisingly, on the last dataset being an extension of the second one utilizing three datasets for training, Random Forest performed best. After its poor performance on the second dataset due to its overfitting, the algorithm was capable of creating a far better generalizing model on the increased number of samples; the reached F-score of 87.1% was even comparable to the result on the third dataset. For Naive Bayes, the magnitude of the drop regarding its performance was comparable to the difference in it between the first pair of datasets. Even if the recurrent neural network performed not that successful concerning the F-score, it seems to be capable of compensating the additional difficulty until a certain extent with the additional data in a way, that the actual difference was not that high like between the first two datasets.

## Part 2: Evaluation on preprocessed data

After the detailed evaluation of the results on the unprocessed data, in the next step, the different preprocessing algorithms were additionally considered into the analysis. Due to their nature as independent parts of a data processing pipeline, the different preprocessing steps were added as optimizable hyperparameter. Using this approach, the actual evaluation becomes again a grid search, this time over the 128 permutations of activated and deactivated steps. As a starting point for this analysis, the optimal set of parameters gained in the experiment before was used. While this dependency on previous results is slightly risky and a clear drawback due to the possibility of another optimum, the exponentially rising number of permutations in combination with the limited available computational power especially in the training of neural networks required such kind of simplification.

| Type  | Dataset       | F-Score | Variance | Precision | Recall |
| ----- | ------------- | ------- | -------- | --------- | ------ |
| Bayes | Single        | 97.2%   | +/- 4.3% | 97.2%     | 97.1%  |
|       | Splitted      | 93.6%   | -        | 92.1%     | 89.1%  |
|       | Multi         | 93.6%   | +/- 1.9% | 93.4%     | 93.3%  |
|       | MultiSplitted | 88%     | -        | 88.7%     | 86.6%  |
| RF    | Single        | 98.2%   | +/- 2.5% | 98.4%     | 98.3%  |
|       | Splitted      | 97%     | -        | 95.4%     | 96.4%  |
|       | Multi         | 96.4%   | +/- 1.7% | 96.4%     | 96.4%  |
|       | MultiSplitted | 94%     | +/- 0.9% | 94%       | 93.9%  |
| RNN   | Single        | 92.7%   | +/- 0.9% | 93.0%     | 92.9%  |
|       | Splitted      | 95.1%   | +/- 1.4% | 92.9%     | 93.2%  |
|       | Multi         | 95.5%   | +/- 0.6% | 95.5%     | 95.5%  |
|       | MultiSplitted | 94.1%   | +/- 1.4% | 94.5%     | 93.7%  |

In consideration of the performances in the first experiment, the actual importance of the preprocessing got evident. For all classifier, the performance was at least improved by 10%, while the recurrent neural network benefited the most. With an average performance of 96.40%, the Random Forest algorithm performed again best, this time followed by the RNN with 94.35% performance. The Naive Bayes implementation was mostly unsuccessful in gaining performance and performed with 93.10%. Besides this increased ability of model generation, a factor worth mentioning is the influence of preprocessing on the variance of the algorithms. Again, the differences between the three different evaluations got reduced for all algorithms, and again mostly beneficial for the recurrent neural network. By this notable change, the algorithm evolves from the "problem child" of the experiment into one of the stablest algorithms, leaving even Naive Bayes behind it. This development is not only advantageous regarding the suitability for a usage in productive contexts but also offers additional confidence regarding a credible interpretation of the results.

Even if most of the notable correlations and conclusions between the different results remains the same in comparison with the first experiment, other results show clear differences. As an example, the two machine learning algorithms used as a baseline for the evaluation of the recurrent neural network performed on the preprocessed data best on the small dataset evaluated with a 3-fold cross-validation; the 98.20% performance reached by the Random Forest algorithm is even the best value gained in the entire experiment. It is not trivial to explain, why an increase of data did not lead to a better model like it would be assumed. One explanation may be, that the preprocessing of the data biased the process of vectorization towards inner features of a single dataset in some way. Unlike these two algorithms, the RNN behaved in entirely different manner: While its performance on the single dataset was not comparable to the performance of the other algorithms, it was the only able to improve its performance on the second, split dataset requiring a more generalized model. Again, an explanation of this effect is not trivial: It seems, that instead of overfitting the data, the RNN generated on the small number of preprocessed samples a too general model being able to show its strength only if it is applied to the completely unknown data. 
On the other two bigger datasets, such a correlation was no longer evident. Common for all algorithms was the drop of performance between the third and the fourth dataset. While these decreases were dramatically for Naive Bayes which falls from 93.60% to 88%, the subtle changes in the performance of the RNN made it to the slightly best performing algorithm on the last dataset.

| Type  | Dataset       | Standart. | Slang | Lemma | Stopword | Stemming | Lower | Punctation     |
| ----- | ------------- | :-------------: | :---: | :---: | :------: | :------: | :-------: | :----: |
| Bayes | Single        |        x        |       |       |          |    x     |           |      |
|       | Splitted      |        x        |       |       |    x     |          |           | x    |
|       | Multi         |        x        |   x   |   x   |          |    x     |     x     |      |
|       | MultiSplitted |        x        |   x   |       |          |          |           |      |
| RF    | Single        |        x        |   x   |       |          |    x     |           | x    |
|       | Splitted      |        x        |   x   |       |    x     |          |           | x    |
|       | Multi         |        x        |   x   |       |    x     |    x     |     x     |      |
|       | MultiSplitted |        x        |   x   |   x   |    x     |          |     x     |      |
| RNN   | Single        |        x        |   x   |   x   |    x     |          |     x     | x    |
|       | Splitted      |        x        |   x   |       |    x     |    x     |     x     | x    |
|       | Multi         |        x        |   x   |   x   |    x     |    x     |     x     | x    |
|       | MultiSplitted |        x        |   x   |   x   |    x     |    x     |     x     |      |

The actual evaluation of the optimal set of the parameters shows some commonalities and some differences: Unquestionably evident is the actual import role of the standardization of the input by a replacement of URLs, numbers, and emoticons. This notable enrichment and normalization of words with semantic meaning is also performed by the removal of spelling mistakes and common slang, too. 
While the probabilistic model of Multinomial Naive Bayes seems to be able to handle stopwords itself, for both other algorithms the removal of that bloat seems to be significant. A context insensitive capitalization of words seems not that important; only the neural network relies on that. A possible explanation might be the removal of slang performed before, which already reduced the number of words differing in their capitalization significantly. The combination of Lemmatization and Stemming seems to be meaningful only for complex datasets with the recurrent neural network; the other algorithms seems to require typically just one of them. Most of the time, even the computational far less expensive Stemming algorithm seems to be entirely sufficient.
To summarize the results, one may claim that the recurrent neural network works undoubtedly best when applied which as preprocessed and standardized data as possible, not only evident due to the increase of performance and the decrease of variance. For the other algorithms applies the well-known rule, that finding the actual optimal preprocessing pipeline is a matter of experimentation. 

# Evaluation and further research

Are recurrent neuronal networks capable of significantly enhancing the way we deal with Spam? According to the result mentioned above, it is trivial to claim the performance of the Recurrent Neural Networks in the classification of spam are not significantly better than other algorithms. In comparison with approaches only taking into the account the occurrences of words and not the semantic structure of sentences, the additional semantic features seem to have no positive influence on the general performance. While this result may not be precisely the result one may wish, some of the conclusions we might draw from the experiment may have relevance for further research.

First of all, Recurrent Neural Networks seem to be highly depended on the number of available samples. In the experiments, an increase in sample size resulted clearly in an increase of performance and reduction of an otherwise uncontrollable variance. To further evaluate the performance of recurrent neural networks, it would, therefore, be highly adequate to apply it to bigger datasets. Even if such resources exist, further preprocessing like a limitation towards the English language would be necessary. While also other algorithms like Random Forest and Naive Bayes profit from the additional data, they seem to be able to compensate a smaller number of training samples quite well. 

Secondly, Recurrent Neural Networks needs extensive preprocessing. Even if deep learning technology is known for being capable of dealing even with contradicting data [@holmes2014], the data itself needs to be in a relatively ordered state. At least with the amount of data used in the experiment, the baseline algorithms seem to be far more robust when working on unprocessed data.

Thirdly, the amount of time the complex neural networks needs to train are even in our times of exponential raising computer power not trivial. On the first glance, it seems acceptable to train a neuronal network twenty minutes on a relatively small dataset of just 2000 samples. However, even a 3-fold validation gets in such situations extremely inefficient, further optimizations of hyperparameters become merely impossible. While additional research may apply the same code just to more powerful hardware to solve such a problem, it seems hard to believe that the scaling of computational power will scale with the computational complexity of deep learning techniques forever. If other algorithms are capable of generating similar results in a fraction of that time, we should solely for the sake of efficiency take care of the choice of our algorithms.

Finally, as an overall conclusion, the experiment showed again unquestionably that there is no silver bullet for any task in machine learning or data mining. Even if we are nowadays able to archive result with our deep learning technology we were unable to imagine only a few years ago, we have to critically evaluate the performance and the efficiency of our tools over and over again. "For a man with a hammer, everything looks like a nail" - according to this slogan it remains necessary keeping our open-minded, unbiased view towards our available tools to reach our goals efficiently and optimally.


# Appendix
## Data - Setups

In [1]:
from abc import ABC
import glob
import csv

from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import f1_score, make_scorer

class Data(ABC):
    def __init__(self, x_train, y_train, x_test, y_test, split_important=True):
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        self.split_important = split_important
    
    def training_size(self):
        return len(self.x_train)
    
    def testing_size(self):
        return len(self.x_test)
    
    def find_optimum(self, pipeline, parameters):
        grid = GridSearchCV(
            pipeline, 
            n_jobs = 1, 
            verbose = 1, 
            scoring = {
                'f1': make_scorer(f1_score, pos_label=True),
                'precision': 'precision_macro',
                'recall': 'recall_macro'
            }, 
            refit = 'f1', 
            cv=(VariationGenerator(self, 3) if self.split_important else None), 
            param_grid=parameters
        )

        print("Training: {} (Training size: {}, Testing size: {})".format(
            self.__class__.__name__, 
            self.training_size(), 
            self.testing_size()
        ))
              
        grid.fit([x for x in self.x_train + self.x_test], 
                 [y for y in self.y_train + self.y_test])
        
        MSG = "{0:1.3f} (+/-{1:1.3f}; Precision: {2:1.3f}, Recall: {3:1.3f}) for {4}"
        for mean, std, prec, recall, params in sorted(
            zip(grid.cv_results_['mean_test_f1'], 
                grid.cv_results_['std_test_f1'],
                grid.cv_results_['mean_test_precision'],
                grid.cv_results_['mean_test_recall'],
                grid.cv_results_['params'],
            ), key = lambda x: x[0], reverse = True)[:3]:

            print(MSG.format(
                mean, 
                std * 2, 
                prec, 
                recall, 
                params
            ))
        
        return grid.best_params_
    
    @staticmethod    
    def _load_file(file):
         with open(file, encoding='ascii', errors='ignore') as spam:
            for row in csv.DictReader(spam, 
                                      delimiter=',', 
                                      quotechar='"', 
                                      skipinitialspace=True, 
                                      strict=True):
                yield [row['CONTENT'].strip(), row['CLASS'] == '1']
                    
class VariationGenerator:
    def __init__(self, data, n):
        self.n = n
        self.training_size = data.training_size()
        self.testing_size = data.testing_size()

    def split(self, *_):
        for _ in range(self.n):
            yield (
                shuffle(list(range(self.training_size))), 
                shuffle(list(range(self.training_size, 
                                   self.training_size + self.testing_size)))
            )
            
    def get_n_splits(self, *_):
        return self.n
        
class SingleFile(Data):
    def __init__(self, path, test_ratio=0.3):
        data = [comment for comment in Data._load_file(path)]
        x_train, x_test, y_train, y_test = train_test_split(
            [comment[0] for comment in data], 
            [comment[1] for comment in data], 
            test_size=test_ratio
        )
        super().__init__(x_train, y_train, x_test, y_test, False)
        
    
class SplittedFile(Data):
    def __init__(self, training, testing, test_ratio=0.3):
        training = [comment for comment in Data._load_file(training)]
        testing = [comment for comment in Data._load_file(testing)]
        super().__init__(
            [comment[0] for comment in training],
            [comment[1] for comment in training],
            [comment[0] for comment in testing[:int(len(training) * test_ratio)]],
            [comment[1] for comment in testing[:int(len(training) * test_ratio)]],
            True
        )

class MixedFiles(Data):
    def __init__(self, paths, test_ratio=0.3):
        data = []
        for file in glob.iglob(paths):
            data.extend(comment for comment in Data._load_file(file))
        x_train, x_test, y_train, y_test = train_test_split(
            [comment[0] for comment in data], 
            [comment[1] for comment in data], 
            test_size=test_ratio
        )
        super().__init__(x_train, y_train, x_test, y_test, False)
        
class SplittedMixedFiles(Data):
    def __init__(self, paths, test_ratio=0.3):
        files = glob.glob(paths)
        shuffle(files)
        
        testing = [comment for comment in Data._load_file(files[0])]
        training = []
        for file in files[1:]:
            training.extend(comment for comment in Data._load_file(file))
            
        super().__init__(
            [comment[0] for comment in training],
            [comment[1] for comment in training],
            [comment[0] for comment in testing],
            [comment[1] for comment in testing],
            True
        )

## Preprocessing

In [2]:
from abc import ABC, abstractmethod
from sklearn.base import BaseEstimator, TransformerMixin

from collections import OrderedDict
import re
import inspect

from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.casual import TweetTokenizer
from nltk.corpus import wordnet, stopwords
from nltk.stem.porter import PorterStemmer

class Preprocessor(ABC):
    @abstractmethod
    def optimize(self, tokenized_comment):
        raise NotImplementedError()
        
    @staticmethod    
    def preprocess(comments, preprocessors):
        tokenizer = TweetTokenizer()
        html_cleaner = re.compile('<.+?>')
        for comment in comments:
            comment = html_cleaner.sub('', comment)
            tokenized_comment = tokenizer.tokenize(comment)
            for preprocessor in preprocessors:
                tokenized_comment = preprocessor.optimize(tokenized_comment)
            yield tokenized_comment

class StandardizePreprocessor(Preprocessor):
    def __init__(self):
        self.regex_url = re.compile(r'[^\s|^\.]+\.[a-z]{2,3}[^\s]*')
        self.regex_number = re.compile(r'\b[0-9]+\b')
        self.regex_emoji = re.compile(r'[\S]{0,3}:[\S]{1,3}')
        self.regex_special = re.compile(r'&[a-z]+;')
        
    def optimize(self, tokenized_comment):   
        return [self.regex_emoji.sub('EMOJII', 
                self.regex_number.sub('NUM', 
                self.regex_url.sub('URL', 
                self.regex_special.sub('', word)))) 
                for word in tokenized_comment]

class SlangPreprocessor(Preprocessor):
    def __init__(self, normalisation_dictionary):
        self.double_character = re.compile(r'(.)\1{2,}')
        
        self.dictionary = {}
        with open(normalisation_dictionary, encoding='ascii', errors='ignore') as f:
            for line in f:
                key, value = line.strip().split("\t")
                self.dictionary[key] = value
            
    def optimize(self, tokenized_comment):
        output = []
        for word in tokenized_comment:
            word = self.double_character.sub(r'\1\1', word)
            if word.lower() in self.dictionary:
                word = self.dictionary[word.lower()]
            output.append(word)
        output = list(OrderedDict.fromkeys(output))
        return output
    
class PosLemmatizationPreprocessor(Preprocessor):
    def __init__(self):
        self.regex_non_word = re.compile(r"[^a-zA-Z\.!?']")
        self.lemmatizer = WordNetLemmatizer()
    
    @staticmethod
    def _tag_to_wordnet(tag):
        if tag.startswith('J'):
            return wordnet.ADJ
        elif tag.startswith('V'):
            return wordnet.VERB
        elif tag.startswith('N'):
            return wordnet.NOUN
        elif tag.startswith('R'):
            return wordnet.ADV
        else:
            return None
    
    def optimize(self, tokenized_comment):
        output = []
        for word in tokenized_comment:
            word = self.regex_non_word.sub('', word).strip()
            if len(word) > 0:
                output.append(word)
                
        for i, (word, tag) in enumerate(pos_tag(output)):
            pos_type = self._tag_to_wordnet(tag)
            if pos_type is not None:
                output[i] = self.lemmatizer.lemmatize(word, pos=pos_type)
            else:
                output[i] = word
                
        return output

class StemmerPreprocessor(Preprocessor):
    def __init__(self):
        self.porter = PorterStemmer()
        
    def optimize(self, tokenized_comment):
        return [self.porter.stem(word) for word in tokenized_comment]
    
class StopwordPreprocessor(Preprocessor):
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        
    def optimize(self, tokenized_comment):
        return [word for word in tokenized_comment 
                if word.lower() not in self.stop_words]
    
class LowercasePreprocessor(Preprocessor):       
    def optimize(self, tokenized_comment):
        return [word.lower() for word in tokenized_comment]

class PunctationRemover(Preprocessor):
    def __init__(self):
        self.char_only = re.compile(r'[^a-zA-Z]')
        
    def optimize(self, tokenized_comment):
        output = []
        for word in tokenized_comment:
            word = self.char_only.sub('', word)
            if len(word) > 0:
                output.append(word)
        return output
    
class PreprocessorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self,
                use_standartize=True,
                use_slang=True,
                use_stopword=True,
                use_lemmatization=True,
                use_stemmer=True,
                use_lowercase=True,
                use_punctation=True):
        args, _, _, values = inspect.getargvalues(inspect.currentframe())
        values.pop("self")

        for arg, val in values.items():
            setattr(self, arg, val)
    
    def fit(self, X, y=None):
        pass
    
    def transform(self, X, y=None):
        preprocessors = []
        if self.use_standartize:
            preprocessors.append(StandardizePreprocessor())
        if self.use_slang:
            preprocessors.append(SlangPreprocessor('dictionaries/slang.txt'))
        if self.use_stopword:
            preprocessors.append(StopwordPreprocessor())
        if self.use_lemmatization:
            preprocessors.append(PosLemmatizationPreprocessor())
        if self.use_stemmer:
            preprocessors.append(StemmerPreprocessor())
        if self.use_lowercase:
            preprocessors.append(LowercasePreprocessor())
        if self.use_punctation:
            preprocessors.append(PunctationRemover())
        return [tokenized for tokenized in Preprocessor.preprocess(X, preprocessors)]
    
    def fit_transform(self, X, y=None):
        return self.transform(X, y)

## Custom "Bag of Words"

In [3]:
from collections import Counter
from sklearn.base import BaseEstimator, TransformerMixin

class BagOfWords(BaseEstimator, TransformerMixin):
    def __init__(self, min_occurrences=2, max_features=None):
        self.counter = Counter()
        self.min_occurrences = min_occurrences
        self.max_features = max_features
        self.bow = None
        
    def fit(self, X, y = None):
        for x in X:
            self.counter.update(x)
            
        self.bow = {}
        i = 2
        for word, occurences in self.counter.most_common(self.max_features):
            if occurences >= self.min_occurrences:
                self.bow[word] = i
                i += 1
    
    def transform(self, X, y = None):
        if self.bow is None:
            raise RuntimeError("Fitting required before transform!")
        
        output = []
        for x in X:
            output.append([self.bow[word] if word in self.bow else 1 for word in x])
        return output
    
    def fit_transform(self, X, y=None):
        self.fit(X)
        return self.transform(X)
    
    def size(self):
        return len(self.bow)

## Recurrent neural network

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import f1_score, accuracy_score

from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, SimpleRNN, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences

import numpy as np

class RnnClassifier(BaseEstimator, ClassifierMixin):  

    def __init__(self, input_length=32, 
                 embedding_dimension=32, 
                 batch_size=32, 
                 epochs=3, 
                 num_hidden_neurons=100,
                 dropout=0,
                 rnn_type='gru',
                 num_words=2000):
        
        self.input_length = input_length
        self.embedding_dimension = embedding_dimension
        self.batch_size = batch_size
        self.epochs = epochs
        self.num_hidden_neurons = num_hidden_neurons
        self.dropout = dropout
        self.rnn_type = rnn_type
        self.num_words = num_words
        self._rnn = None
        
    def fit(self, X, y=None):
        assert (y is not None), "Y is required"
        assert (self.rnn_type in ['gru', 'lstm', 'simple']), "Invalid RNN type"
        
        X = pad_sequences(X, self.input_length)
        X = np.clip(X, 0, self.num_words - 1)
        
        self._rnn = Sequential()
        self._rnn.add(Embedding(self.num_words, 
                                self.embedding_dimension, 
                                input_length=self.input_length)
                     )
        
        if self.dropout > 0:
            self._rnn.add(Dropout(self.dropout))
        
        if self.rnn_type is 'gru':
            self._rnn.add(GRU(self.num_hidden_neurons))
        elif self.rnn_type is 'lstm':
            self._rnn.add(LSTM(self.num_hidden_neurons))
        else:
            self._rnn.add(SimpleRNN(self.num_hidden_neurons))
            
        if self.dropout > 0:
            self._rnn.add(Dropout(self.dropout))
        self._rnn.add(Dense(1))
        self._rnn.add(Activation('sigmoid'))
        
        self._rnn.compile(loss='binary_crossentropy', 
                          optimizer='adam', 
                          metrics=['accuracy'])
        
        self._rnn.fit(X, y, epochs=self.epochs, 
                      batch_size=self.batch_size, 
                      verbose=0)
        
        return self

    def predict(self, X, y=None):
        if self._rnn is None:
            raise RuntimeError("Fitting required before prediction!")

        X = pad_sequences(X, self.input_length)
        return [prob[0] >= 0.5 for prob in 
                self._rnn.predict(X, batch_size=self.batch_size)]

    def score(self, X, y=None):
        assert (y is not None), "Y is required"
        
        prediction = self.predict(X)
        return f1_score(y, prediction)

## Experiment

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

def dummy(x):
    return x

# Create the necessary helper
preprocessor = PreprocessorTransformer(use_standartize=False, 
                                       use_slang=False, 
                                       use_stopword=False, 
                                       use_lemmatization=False,
                                       use_stemmer=False,
                                       use_lowercase=False,
                                       use_punctation=False)

vectorizer = CountVectorizer(tokenizer=dummy, preprocessor=dummy, max_features=2000)

preprocessor_setup = {
    "pre__use_standartize": [True, False],
    "pre__use_slang": [True, False],
    "pre__use_stopword": [True, False],
    "pre__use_lemmatization": [True, False],
    "pre__use_stemmer": [True, False],
    "pre__use_lowercase": [True, False],
    "pre__use_punctation": [True, False]
}

# Create the pipelines
rnn_pipeline = Pipeline([
    ("pre", preprocessor),
    ("bow", BagOfWords(max_features=2000)),
    ("rnn", RnnClassifier())
], memory='cache')

bayes_pipeline = Pipeline([
    ("pre", preprocessor),
    ("vectorizer", vectorizer),
    ("bayes", MultinomialNB())
], memory='cache')

forest_pipeline = Pipeline([
    ("pre", preprocessor),
    ("vectorizer", vectorizer),
    ("forest", RandomForestClassifier())
], memory='cache')
 
# Load the datasets
datasets = [
    SingleFile('data/Youtube01-Psy.csv'),
    SplittedFile('data/Youtube01-Psy.csv', 'data/Youtube02-KatyPerry.csv'),
    MixedFiles('data/*.csv'),
    SplittedMixedFiles('data/*.csv')
]

# Start the experiment
print("Naive Bayes:")
for data in datasets:
    optimum = data.find_optimum(bayes_pipeline, {
        "bayes__alpha": [0.5, 0.75, 1.0, 1.25, 1.5],
        "bayes__fit_prior": [True, False],
    })
    
    print("-> Preprocessors:")
    bayes_pipeline.set_params(**optimum)
    data.find_optimum(bayes_pipeline, preprocessor_setup)
    
print("\nRecurrent neural network:")
for data in datasets:
    optimum = data.find_optimum(rnn_pipeline, {
        "rnn__epochs": [3, 10, 20],
        "rnn__num_hidden_neurons": [50, 100, 200],
        "rnn__dropout": [0, 0.1, 0.2],
        "rnn__rnn_type": ['gru', 'lstm', 'simple']
    })
    
    print("-> Preprocessors:")
    rnn_pipeline.set_params(**optimum)
    data.find_optimum(rnn_pipeline, preprocessor_setup)

print("\nRandom Forest:")
for data in datasets:
    optimum = data.find_optimum(forest_pipeline, {
        "forest__n_estimators": [10, 100, 500, 800],
        "forest__max_features": ['sqrt', 'log2', None],
    })
    
    print("-> Preprocessors:")
    forest_pipeline.set_params(**optimum)
    data.find_optimum(forest_pipeline, preprocessor_setup)

# Bibliography